# Framework Modulaire de Reconnaissance de Signes (SLR)

Ce notebook démontre l'utilisation de l'architecture modulaire `src/`.  
Il permet de comparer facilement différentes stratégies de **prétraitement** (ex: segmentation de peau) et différentes **architectures de modèles**.

Cette architecture est **agnostique** au dataset utilisé. Elle peut être configurée pour l'alphabet ASL, LSF, ou tout autre dataset d'images classées par dossiers.

## Structure du Projet
- `src.config`: Configuration centralisée (supporte plusieurs environnements)
- `src.preprocessing`: Algorithmes de segmentation (HSV, etc.)
- `src.data`: Pipeline de chargement dynamique
- `src.model`: SignCNN (Modèle Générique) et MobileNetV2
- `src.train`: Moteur d'entraînement

In [ ]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import cv2
import numpy as np

# Import des modules locaux
from src.config import config
from src.data import get_dataloaders
from src.model import get_model, SignCNN
from src.train import Trainer
from src.preprocessing import HSVSkinSegmenter, NoOpPreprocessor

print(config)

## 1. Choix de la Stratégie de Segmentation
C'est ici que réside la modularité du prétraitement. Vous pouvez choisir entre :
- `NoOpPreprocessor`: Aucune segmentation (Baseline)
- `HSVSkinSegmenter`: Segmentation par couleur de peau HSV

In [ ]:
# CHOIX DE L'EXPÉRIENCE
USE_SEGMENTATION = True

if USE_SEGMENTATION:
    print("✅ Activation de la Segmentation HSV (Peau)")
    # Vous pouvez ajuster les seuils HSV ici si nécessaire
    preprocessor = HSVSkinSegmenter(lower_hsv=[0, 20, 70], upper_hsv=[20, 255, 255])
else:
    print("❌ Pas de segmentation (Raw Images)")
    preprocessor = NoOpPreprocessor()

## 2. Chargement des Données
Le `preprocessor` est injecté dans le pipeline de transformation.

In [ ]:
# Vous pouvez surcharger config.DATA_DIR ici si vous utilisez un autre dataset (ex: LSF)
# config.DATA_DIR = "/chemin/vers/lsf_dataset"

train_loader, val_loader, class_names = get_dataloaders(preprocessor=preprocessor)
config.NUM_CLASSES = len(class_names)

### Visualisation des Données (Après Preprocessing)
Vérifions à quoi ressemblent les images qui entrent dans le réseau.

In [ ]:
import torchvision

def imshow(inp, title=None):
    """Affichage d'un tenseur image"""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title:
        plt.title(title)
    plt.axis('off')

# Récupération d'un batch
inputs, classes = next(iter(train_loader))
out = torchvision.utils.make_grid(inputs[:4])

plt.figure(figsize=(10, 5))
imshow(out, title=[class_names[x] for x in classes[:4]])
plt.show()

## 3. Configuration du Modèle et Entraînement
Choix possible : `'custom'` (SignCNN) ou `'mobilenet_v2'`.

In [ ]:
model = get_model(model_name="custom", num_classes=config.NUM_CLASSES)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config.LEARNING_RATE)

trainer = Trainer(model, {'train': train_loader, 'val': val_loader}, criterion, optimizer)

# Entraînement
model, history = trainer.train(epochs=config.EPOCHS)
trainer.save_model("final_model.pth")

## 4. Analyse des Performances

In [ ]:
# Courbes d'apprentissage
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history['train_loss'], label='Train Loss')
plt.plot(history['val_loss'], label='Val Loss')
plt.legend()
plt.title("Loss Evolution")

plt.subplot(1, 2, 2)
plt.plot(history['train_acc'], label='Train Acc')
plt.plot(history['val_acc'], label='Val Acc')
plt.legend()
plt.title("Accuracy Evolution")
plt.show()

# Matrice de Confusion
print("Génération de la Matrice de Confusion...")
all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(config.DEVICE)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())

plt.figure(figsize=(12, 10))
cm = confusion_matrix(all_labels, all_preds)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.ylabel('Vraie Classe')
plt.xlabel('Classe Prédite')
plt.show()

print(classification_report(all_labels, all_preds, target_names=class_names, labels=range(len(class_names))))